# On Assessing Your Environment
---

## Overview

Over the course of the YGL Workshop at Princeton, you will learn how we estimate, quantify, and classify and optimize energy globally and locally. A necessary aspect for this is measuring what we can as consistently and accurately as possible. 

In Tuesday's lecture we will discuss is some detail how we measure and estimate energy consumption as well as how we establish **set points** (or operational targets) around which to minimize energy consumption.

In the US, approximately 1/3rd of all energy (fuel and electricity) is consumed in maintaining a comfortable environment through heating, ventilation, and air conditions (HVAC). This is not a trivial quantity of energy. 

In this introduction, you will learn a few ways in which building conditions can be measured, and you will learn how low-cost microcontrollers are used to convert physically quantities in to digital information that can be shared, stored, and analyzed for real-time and collective decision making.


## How We Achieve Comfort

In the world of HVAC, there are four gross classes of devices

- Actuators
- Sensors
- Controllers
- Passives

**Actuators** are the elements on which most energy is spent, and these are the boilers, air conditioners, fans, space heaters etc. These devices inherently have no sense of what the environment is around them, they just convert energy to change the local environment (temperature, humidity, airflow, etc).

**Sensors** are then required to provide information that can be used to inform the actuation. Sensors are used to measure temperature, humidity, light, airflow, occupancy, etc.

The sensor and the actuator are inherently "dumb". A **controller** is used to make decisions based on the sensor reading to determine when and how the actuators should behave. The controller typically a computer program running on a thermostat or within a specialize CPU within a building.

Finally, **passives** are objects which do not sense, actuate, nor controller but rather physically bound the environment. Passives include walls, ceilings, insulation, clothes, anything that can be used to define an "inside" and "outside". Passives are critical for physically delimiting the space a sensor, an actuator and controller will work to define comfort.

## How We Sense

### A Computer Understands Switches
Just as a computer has no inherent concept if comfort, it has no inherent sense of temperature, humidity, light, or sound. A computer, as you might have heard, understand `0` and `1` and binary combinations thereof, and when interacting with the physical world, what defines `0` or `1` is electrical potential, or **voltage**. It also counts time very well.

At its core, a `0` or `1` is the base unit of measurement, and this choice is called a **bit**. While the world is full of important binary aspects like light and dark, cold or hot, yes or no, we are typically most comfortable between extremes. 

Computers can do nothing but remember and count, but they are really good at both, so we can chain bits together to get better levels of discretion. 

So if a one bit sensor tells me whether something is

1. Bright or 
2. Not Bright

a two-bit sensor might be able to distinguish

1. Very Bright
2. Pretty Bright
3. Sort of Bright
4. Not Bright 

We see above that bits operate in powers of 2, such that

$$ a = 2^{n}$$

where $a$ is the levels of discretion and $n$ is the number of bits chained together. So three bits provides eight levels of discretion, eight bits provides 256 levels of discretion, 12 bits provides 4096 levels of discretion, and so on.

The real world is **infinitely and infinitesimally analog**, so depending on how we wire our sensor and our bit wise sensitivity, the infinite possibilities temperature can be in reality are bound to and binned within a finite number of measurements within the computer.

### Back To Voltage

But before we can understand light, temperature and humidity from bits, we need to convert light, temperature humidity into a voltage, and then that voltage into a bit.

Starting backwards, the computer sets limits on what `0` (off) and `1` (on) are in terms of voltage, and this voltage is defined by a combination of the power that computer operates on and how the power conditioned. 

While there are a large number of operating voltage standards, a modern and ubiquitous standard is the USB minimum supply voltage of 5 V. On most small devices (e.g. phones), 5 V is the maximum potential anything within the phone sees, and the USB port provides this potential. 

But the USB port only needs to provide 5 V +/- 5%. This is just fine for power, but it can make sensor readings a bit challenging if absolute voltage is required. For this reason, and reasons of chip standardization, on modern small devices the sort of 5 V power line is stepped down to a very firm 3.3 V, so 3.3 V defines our `1`. Our `0` is the common ground of the USB device. 

Based on our binning above, for a 1 bit sensor, anything that is higher than of 1.6 V (3.3V/2) will generally be binned "on", and anything below 1.6 V is generally binned to "off".

Luckily, the devices we will be using to measure things have 12 bit analog to digital (ADC), so 0 is 0V and 3.3V is 4095. 

So now we can measure voltage, how can we convert this physical measurement to this voltage?

We can invoke Ohm's law, where 

$$ V = I Z$$

and $I$ is the current and $Z$ is the impedance. 

Current represents the flow of charge through the circuit, and the impedance is anything that tries to work against that flow. Impedance can be

- Resistance, normally denoted with $R$,
- Capacitance, normally denoted with $C$ 
- Inductance, normally denoted with $L$

Since Ohm's law defines all electrical circuits, sensors that convert, or transduce, between the physical and electrical domains must involve a physical dependent, voltage, current, or impedance, or some combination of the three. When the physically dependent quantity is capacitance or inductance, time must be measured carefully as well. Fortunately computers measure time _very_ well, with a GHz class microcontroller, by definition, being able to measure time below nanoseconds.

But for our study, we're going to just use a temperature dependent resistor, or a thermistor, and design a simple circuit that converts the change in resistance to a measurable change in potential, and we can write

$$V = IR$$

The problem is that our microcontroller of choice, as with most microntollers, source voltage, not current. So if we can source voltage, and measure voltage, we need a way to link the source and the measurement without directly tying those points together. 

Let's create a voltage divider to do this.

![](https://drops.steingart.princeton.edu/ygl/vd.png)

In the above schematic, we are sourcing voltage at $V_{in}$, and measuring voltage at $V_{out}$, and no current is flowing to $V_{out}$, so the only current path for $I$ is between $V_{in}$ and $GND$.

This means that $I$ is the same everywhere in the circuit, so we can use Ohm's law to state:

$$V_{in}  = I*(R_{1}+R_{2})$$

and

$$V_{out} = I*R_{2}$$

In the above network of equations, we will use our thermistor in the position of $R_1$, and choose a fixed (constant) and known resistor in the position of $R_2$. We will then rearrange the equation above so that $R_1$ is a function of $V_{out}$ (which we will measure with our ADC), so

$$I = \frac{V_{in}}{R_1+R_2} = \frac{V_{out}}{R_2}$$

so

$$R_1 = \left(\frac{V_{in}}{V_{out}}-1\right)*R_2$$

So now we have a way of measuring how $R_1$ changes with temperature, and we can use a calibration table to then determine what the actual temperature is.

Phew.

So let's do it for real.

## Introducing the Breadboard

A solderless breadboard is a manual rapid-protyping toolfor building and testing circuits before we design permanent circuit.

### How Does a Breadboard Work?
The images and descriptions in the section are reproduced directly and verbatim from [science buddies](https://www.sciencebuddies.org/science-fair-projects/references/how-to-use-a-breadboard).

<blockquote>
<p>The leads can fit into the breadboard because the <i>inside</i> of a breadboard is made up of rows of tiny metal clips. This is what the clips look like when they are removed from a breadboard.
</p>

 
<img src="https://d1ca4yhhe0xc0x.cloudfront.net/Files/7318/7/breadboard-clips.jpg" alt="breadboard spring clips" width="250" height="226" />
 


<p>When you press a component's lead into a breadboard hole, one of these clips grabs onto it.
</p>
  
<img src="https://d1ca4yhhe0xc0x.cloudfront.net/Files/7319/7/breadboard-component-clip.jpg" alt="component inserted into breadboard clip" width="250" height="422" />


<p>Some breadboards are actually made of transparent plastic, so you can see the clips inside.
</p>

 
<img src="https://d1ca4yhhe0xc0x.cloudfront.net/Files/7320/6/transparent-breadboard-clips.jpg" alt="transparent breadboard clips" width="500" height="185" />
 
<p>Most breadboards have a backing layer that prevents the metal clips from falling out. The backing is typically a layer of sticky, double-sided tape covered by a protective layer of paper. If you want to permanently "stick" the breadboard to something (for example, a robot), you just need to peel off the paper layer to expose the sticky tape underneath. In this picture, the breadboard on the right has had its backing removed completely (so you can see all the metal clips). The breadboard on the left still has its sticky backing, with one corner of the paper layer peeled up.
</p>
 
 
<img src="https://d1ca4yhhe0xc0x.cloudfront.net/Files/7321/7/breadboard-backing.jpg" alt="breadboard adhesive backing" width="500" height="388" />

<p>Most breadboards have some numbers, letters, and plus and minus signs written on them. What does all that mean? While their exact appearance might vary from breadboard to breadboard, the general purpose is always the same. These labels help you locate certain holes on the breadboard so you can follow directions when building a circuit. If you have ever used a spreadsheet program like Microsoft Excel&reg; or Google Sheets&trade;, the concept is exactly the same. Row numbers and column letters help you identify individual holes in the breadboard, just like cells in a spreadsheet. For example, all of the highlighted holes are in "column C."
</p>

 
<img src="https://d1ca4yhhe0xc0x.cloudfront.net/Files/7322/6/breadboard-column-C.png" alt="breadboard column C" width="300" height="469" />

<p>All of the highlighted holes are in "row 12."
</p>
 
<img src="https://d1ca4yhhe0xc0x.cloudfront.net/Files/7323/6/breadboard-row-12.png" alt="breadboard row 12" width="300" height="469" />
  
<p>"Hole C12" is where column C intersects row 12.</p> 

 
<img src="https://d1ca4yhhe0xc0x.cloudfront.net/Files/7324/6/breadboard-hole-C12.png" alt="breadboard hole C12" width="300" height="469" />
  
<p>What about the long strips on the side of the breadboard, highlighted in yellow here?</p>


<img src="https://d1ca4yhhe0xc0x.cloudfront.net/Files/7325/6/power-bus-highlights.png" alt="breadboard power and ground buses" width="300" height="469" />
 
<p>These strips are typically marked by red and blue (or red and black) lines, with plus (+) and minus (-) signs, respectively. They are called the <b>buses</b>, also referred to as <b>rails</b>, and are typically used to supply electrical power to your circuit when you connect them to a battery pack or other external power supply. You may hear the buses referred to by different names; for example, <i>power bus</i>, <i>positive bus</i>, and <i>voltage bus</i> all refer to the one next to the red line with the plus (+) sign. Similarly, <i>negative bus</i> and <i>ground bus</i> both refer to one next to the blue (or black) line with the minus (-) sign. Sound confusing? Use this table to help you remember&mdash;there are different ways to refer to the buses, but they all mean the same thing. Do not worry if you see them referred to by different names in different places (for example, in different Science Buddies projects or other places on the internet). Sometimes you might hear "power buses" (or rails) used to refer to <i>both</i> of the buses (or rails) together, not just the positive one.
</p>

<table>
<thead>
<tr><th>Positive</th>	<th>Negative</th></tr>
</thead>
<tbody>
<tr><td>Power</td>	<td>Ground</td></tr>
<tr><td>Plus sign (+)</td>	<td>Minus sign (-)</td></tr>
<tr><td>Red</td>	<td>Blue or black</td></tr>
</tbody>
</table>


<p>Note that there is no physical difference between the positive and negative buses, and using them is not a requirement. The labels just make it easier to organize your circuit, similar to <a href="#color-code">color-coding your wires</a>.
</p>

<p>Remember that the inside of the breadboard is made up of sets of five metal clips. This means that each set of five holes forming a half-row (columns A&ndash;E or columns F&ndash;J) is electrically connected. For example, that means hole A1 is electrically connected to holes B1, C1, D1, and E1. It is <i>not</i> connected to hole A2, because that hole is in a different row, with a separate set of metal clips. It is also <i>not</i> connected to holes F1, G1, H1, I1, or J1, because they are on the other "half" of the breadboard&mdash;the clips are not connected across the gap in the middle (to learn about the gap in the middle of the breadboard, see the <a href="#integrated-circuits">Advanced</a> section). Unlike all the main breadboard rows, which are connected in sets of five holes, the buses typically run the entire length of the breadboard (but there are some exceptions). This image shows which holes are electrically connected in a typical half-sized breadboard, highlighted in yellow lines.
</p>


 
<img src="https://d1ca4yhhe0xc0x.cloudfront.net/Files/7326/6/breadboard-row-connections.png" alt="breadboard hole electrical connections" width="300" height="469" />
  


<p>Buses on opposite sides of the breadboard are <i>not</i> connected to each other. Typically, to make power and ground available on both sides of the breadboard, you would connect the buses with jumper wires, like this. Make sure to connect positive to positive and negative to negative (see the section on <a href="#lines">buses</a> if you need a reminder about which color is which).
</p>

 
<img src="https://d1ca4yhhe0xc0x.cloudfront.net/Files/7327/6/power-bus-jumpers.jpg" alt="breadboard power bus jumper wires" width="400" height="287" />

</blockquote>



